# Natural Languange Processing (NLP)- Advanced Topics in DL

Group Z:</br>
    - Iliya Morgunov - 206361412</br>
    - Eadan Schechter - 209793553

# Imports

In [ ]:
#pip install -U transformers

In [ ]:
#pip install transformers[torch]

In [ ]:
#pip install optuna wandb

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

import time
import os
os.environ["WANDB_SILENT"] = "true"
import shutil
import re

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, EarlyStoppingCallback, Trainer
from datasets import Dataset as HFDataset        # Hugging Face

import torch
from torch import nn, optim
from torch.utils.data import Dataset as TorchDataset, DataLoader  # PyTorch
import torch.nn.functional as F
from torch.quantization import quantize_dynamic
from torch.nn.utils import prune

from datetime import datetime
from tqdm import tqdm

import optuna
optuna.logging.set_verbosity(optuna.logging.ERROR)
import wandb
wandb.login(key="a4366556f2db644bb48872fb5da34a12fcdb200d")


import warnings
from transformers.utils import logging as hf_logging

warnings.filterwarnings("ignore")
hf_logging.set_verbosity_error()

In [40]:
from datetime import datetime
import re, os

def _slug(s: str) -> str:
    return re.sub(r'[^A-Za-z0-9_\-]+', '_', s.replace('/', '-')).strip('_')

def set_wandb_project(teacher_model_name: str, student_model_name: str, prefix="KD"):
    date = datetime.now().strftime("%Y%m%d")
    project = f"{prefix}-{_slug(student_model_name)}-from-{_slug(teacher_model_name)}-{date}"
    os.environ["WANDB_PROJECT"] = project
    # (optional but handy for grouping)
    os.environ["WANDB_RUN_GROUP"] = f"{_slug(student_model_name)}-from-{_slug(teacher_model_name)}"
    return project

In [3]:
df_train = pd.read_csv("df_train_final.csv")
df_test = pd.read_csv("df_test_final.csv")

# Part 2

In [4]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())


True
12.4
90300


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Fine-tuning BERTweet

BERTweet is a RoBERTa-based language model specifically pre-trained on over 850 million English tweets to capture the linguistic nuances and informal conventions of social media text. Academic research has shown that BERTweet achieves state-of-the-art results for tweet sentiment analysis. For example, Nguyen et al. (2020) demonstrated that BERTweet substantially outperformed general-domain models like RoBERTa and XLM-R when classifying tweet sentiment, improving the previous state-of-the-art by approximately 5% absolute F1 on the SemEval-2017 Twitter sentiment benchmark. This improvement translates to a significantly higher accuracy in classifying tweets as positive, negative, or neutral, compared to earlier transformer-based approaches. The model’s strong performance is attributed to its large-scale, domain-specific pre-training, which allows it to recognize Twitter slang, emojis, hashtags, and informal syntax much better than generic BERT variants.

The BERTweet pre-training corpus is highly relevant to our own project. It consists of an 80GB collection of uncompressed text containing 850 million tweets (about 16 billion word tokens), gathered from two main sources. The first and largest corpus (845M tweets) was collected from the Twitter Stream Archive, spanning from January 2012 to August 2019, and includes only English-language tweets. These tweets were pre-processed with the TweetTokenizer from the NLTK toolkit, and emojis were converted into text. User mentions and URLs were normalized into special tokens (@USER, HTTPURL), and only tweets containing between 10 and 64 tokens were retained, with retweets excluded. Notably, the second component of the pre-training data includes 5 million English tweets related to the COVID-19 pandemic, collected between January and March 2020—an interval that partially overlaps with the timeline of the data in our own analysis. This overlap suggests that BERTweet is well-equipped to handle the vocabulary, topics, and sentiment expressions specific to the COVID-19 period.

BERTweet is publicly available and can be easily downloaded (e.g., from the Hugging Face Hub as vinai/bertweet-base) and fine-tuned in PyTorch for sentiment classification tasks. In summary, BERTweet represents a best-in-class choice for Twitter sentiment analysis, as it combines the robust RoBERTa architecture with domain knowledge of tweets—including some from the same time window as our dataset—enabling more accurate and contextually aware sentiment modeling.

In [6]:
# Load the BERTweet model and tokenizer and inspect
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)
try:
    model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=5).to(device) # We have 5 sentiments
except:
    model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=5, use_safetensors=True).to(device) # We have 5 sentiments

#model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=5, use_safetensors=True).to(device) # We have 5 sentiments
model

config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

### PyTorch Fine-Tuning

The TweetDataset class inherits from PyTorch’s Dataset class, enabling us to efficiently handle and preprocess tweet data for model training.

This custom dataset class is especially useful for working with data formats *not directly supported by PyTorch's built-in datasets, and text data that requires tokenization and transformation before being fed into a neural network.

The three essential methods are:

__init__: Initializes the dataset with the raw tweet texts, corresponding sentiment labels, tokenizer, and any additional settings.

__len__: Returns the total number of tweet samples in the dataset, which allows PyTorch DataLoader to know how many batches to create.

__getitem__: Retrieves a single tweet (data sample) and its label by index, applies the tokenizer to convert the tweet into input features (input_ids, attention_mask), and returns the tensors ready for model input.

This structure allows for flexible and efficient batching, shuffling, and preprocessing of tweets during model training and evaluation.

We now implement the TweetDataset class and prepare our data for PyTorch training. The sequence length was selected based on our EDA (see previous section), and all data and labels are processed for efficient batching.

For model training, we use a standard fine-tuning pipeline with early stopping, hyperparameter tuning via Optuna, and experiment tracking with Weights & Biases (W&B), following best practices introduced in course Exercise 4.

In [7]:
class TweetDataset(TorchDataset): # Dataset Class
    """
    PyTorch Dataset for tweet sentiment analysis using a DataFrame.
    Processes each row to create tokenized input for transformer models
    """

    def __init__(self, dataframe, tokenizer, text_col="TweetWithDateLocation", label_col="label", max_len=128):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame containing tweets and labels.
            tokenizer: Hugging Face tokenizer instance.
            text_col (str): Column name containing tweet text.
            label_col (str): Column name containing sentiment label (as integer or already encoded).
            max_len (int): Maximum sequence length for model input.
        """

        self.texts = dataframe[text_col].astype(str).tolist()
        self.labels = dataframe[label_col].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len # Ensures all sequences have the same length (Maximum tweet length allowed)

    def __len__(self):
        # Returns total number of tweets
        return len(self.texts)

    def __getitem__(self, idx):
         # Retrieves and processes a single tweet at the given index.

        text = self.texts[idx]     # Raw tweet text
        label = self.labels[idx]        # Corresponding sentiment label

        # Tokenize the tweet with padding/truncation to max_len using the provided tokenizer
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,    # Pad/truncate to this length (Ensures uniform input size)
            padding='max_length',       # Add padding to reach the maximum length (pad shorter tweets to max_len)
            truncation=True,            # Truncate if longer than max_len
            return_attention_mask=True, # Generate attention mask for BERT
            return_tensors='pt'         # Output as PyTorch tensors
        )

        #  Return a dictionary containing tokenized inputs, attention masks, and the label
        return {
            'input_ids': encoding['input_ids'].squeeze(),           # Tensor of size [max_len]
            'attention_mask': encoding['attention_mask'].squeeze(), # Tensor of size [max_len]
            'labels': torch.tensor(label, dtype=torch.long)         # Scalar tensor
        }

Apply Label Encoding to the data set sentiments

In [8]:
# Map Sentiment to 0-4 integers (sentiment columns are now integer incoded)
sentiment_classes = ['Negative', 'Neutral', 'Positive', 'Extremely Negative', 'Extremely Positive']
label2id = {label: idx for idx, label in enumerate(sentiment_classes)}
df_test['label'] = df_test['Sentiment'].map(label2id) # Full test set
df_train['label'] = df_train['Sentiment'].map(label2id) # Full train set

#### Early stopping function

In [9]:
def early_stop_check(patience, best_val_f1, best_val_f1_epoch, current_val_f1, current_epoch):
    early_stop_flag = False # Initialize flag to be False (no need to early stop
    if current_val_f1 > best_val_f1:
        # If we improved the F1, update the parameteres holding the best val f1 details
        best_val_f1 = current_val_f1
        best_val_f1_epoch = current_epoch
    else:
        # No improvement, check for patience: check if there has been more than the acceptable number of epochs where val f1 hasn't improved
        if current_epoch - best_val_f1_epoch > patience:
            early_stop_flag = True # Change flag
    return best_val_f1, best_val_f1_epoch, early_stop_flag

#### Training Function

In [10]:
def train_model_with_hyperparams(model, model_name, train_loader, val_loader, optimizer, criterion, epochs, patience, trial):
    best_val_f1  = 0.0 # Keep track of best f1
    best_val_f1_epoch  = 0
    early_stop_flag = False
    best_model_state = None

    for epoch in range(1, epochs + 1):
        model.train() # Enable training mode
        train_loss = 0.0
        total_train_samples = 0
        correct_train_predictions = 0

        for batch in train_loader: #Iterates over the train_loader, which is a DataLoader object containing batches of training data.
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad() # Reset gradients
            outputs = model(input_ids, attention_mask=attention_mask) # Forward pass
            logits = outputs.logits # save the logits (the raw output of the model)
            loss = criterion(logits, labels) # Calculate loss

            loss.backward() # Backward pass
            optimizer.step() # Update weights using the optimizer

            # Accumulate training loss and predictions
            train_loss += loss.item() * input_ids.size(0)
            total_train_samples += input_ids.size(0)
            correct_train_predictions += (logits.argmax(dim=1) == labels).sum().item()

        train_loss /= total_train_samples
        train_accuracy = correct_train_predictions / total_train_samples

        ###  Validation loop  ###
        model.eval() # Enable evaluation mode
        val_loss = 0.0
        total_val_samples = 0
        correct_val_predictions = 0

        all_val_labels = []
        all_val_preds = []

        with torch.no_grad(): # Disable gradient computation
            for batch in val_loader: # iterate on the val_loader's batches
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                loss = criterion(logits, labels)

                val_loss += loss.item() * input_ids.size(0)
                total_val_samples += input_ids.size(0)
                correct_val_predictions += (logits.argmax(dim=1) == labels).sum().item()

                all_val_labels.extend(labels.cpu().numpy())
                all_val_preds.extend(logits.argmax(dim=1).cpu().numpy())

        # calculate metrics
        val_loss /= total_val_samples
        val_accuracy = correct_val_predictions / total_val_samples
        val_precision = precision_score(all_val_labels, all_val_preds, average='macro')
        val_recall = recall_score(all_val_labels, all_val_preds, average='macro')
        val_f1 = f1_score(all_val_labels, all_val_preds, average='macro')

        if patience is not None:
            # Check for early stopping
            best_val_f1, best_val_f1_epoch, early_stop_flag = early_stop_check(patience, best_val_f1, best_val_f1_epoch, val_f1, epoch)

        # Save the best model under the best_model_state parameter
        if val_f1 == best_val_f1:
            best_model_state = model.state_dict()

        # Log metrics to Weights & Biases
        if wandb.run is not None:
            wandb.log({
                "Epoch": epoch,
                "Train Loss": train_loss,
                "Train Accuracy": train_accuracy,
                "Validation Loss": val_loss,
                "Validation Accuracy": val_accuracy,
                "Validation Precision": val_precision,
                "Validation Recall": val_recall,
                "Validation F1": val_f1})

        if early_stop_flag:  # Checks whether the early stopping condition has been met, as indicated by the early_stop_flag
            break # Exits the training loop immediately if the early stopping condition is satisfied

    if best_model_state is not None:
    # Save the best model if tracked (e.g., via early stopping or best F1)
        if trial is not None:
            torch.save(best_model_state, f"best_model_trial_{trial.number}.pt")
        else:
            torch.save(best_model_state, f"{model_name.replace('/', '-')}_fine_tuned_pytorch_model.pt")
    else:
        # No best state was tracked — save the final model as-is
        torch.save(model.state_dict(), f"{model_name.replace('/', '-')}_fine_tuned_pytorch_model.pt")

    return val_f1

#### Optuna Function & Study

In [11]:
# Objective Function for Optuna
def objective(trial, train_df, tokenizer, model_name, text_col, label_col, device):

    """
    Generic Optuna objective function for HuggingFace transformer-based models (e.g., BERTweet, DeBERTa).
    """

    # Hyperparameter suggestions
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-4)
    patience = trial.suggest_int("patience", 5, 7)
    batch_size = trial.suggest_categorical("batch_size", [32, 64])
    epochs = trial.suggest_int("epochs", 10, 25, step=5)
    num_layers = trial.suggest_int("num_layers", 1, 1)

    # For cross-validation
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    f1_scores = []

    # Extract features and labels as lists (so indices match)
    X = train_df[text_col].tolist()
    y = train_df[label_col].tolist()

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train = [X[i] for i in train_idx]
        y_train = [y[i] for i in train_idx]
        X_val = [X[i] for i in val_idx]
        y_val = [y[i] for i in val_idx]

        train_fold_df = pd.DataFrame({text_col: X_train, label_col: y_train})
        val_fold_df = pd.DataFrame({text_col: X_val, label_col: y_val})

        train_dataset = TweetDataset(train_fold_df, tokenizer, text_col=text_col, label_col=label_col, max_len=128) # Create the TweetDataset object
        val_dataset = TweetDataset(val_fold_df, tokenizer, text_col=text_col, label_col=label_col, max_len=128) # Create the TweetDataset object

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # Insert into a DataLoader
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) # Insert into a DataLoader

        # Initialize a model (generic) for each fold
        try:
            model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)
        except:
            model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, use_safetensors=True).to(device)

        #model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, use_safetensors=True).to(device)

        # Freezing logic for DeBERTa
        if hasattr(model, "deberta") and hasattr(model.deberta, "encoder"):
            # Freeze all DeBERTa encoder parameters
            for param in model.deberta.parameters():
                param.requires_grad = False

            # Unfreeze the last `num_layers` DeBERTa encoder layers
            encoder_layers = model.deberta.encoder.layer
            for layer in encoder_layers[-num_layers:]:
                for param in layer.parameters():
                    param.requires_grad = True

        # Freezing logic for RoBERTa type models (e.g BERTweet)
        elif hasattr(model, "roberta") and hasattr(model.roberta, "encoder"):
            for param in model.roberta.parameters():
                param.requires_grad = False

            encoder_layers = model.roberta.encoder.layer
            for layer in encoder_layers[-num_layers:]:
                for param in layer.parameters():
                    param.requires_grad = True

        # Always unfreeze classifier (for any model applied)
        for param in model.classifier.parameters():
            param.requires_grad = True

        # Define optimizer and loss function
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        # Initialize Weights & Biases logging - the values in the config are the properties of each trial.
        wandb.init(project=f"{model_name.replace('/', '-')} Sentiment Analysis Fine-Tuning Full Code - {datetime.now().date().strftime('%Y%m%d')}",
            config={
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "patience": patience,
            "batch_size": batch_size,
            "num_layers": num_layers,
            "architecture": model_name,
            "dataset": "Corona_NLP"
            },
            name=f"trial_{trial.number}_fold_{fold}", # The name that will be saved in the W&B platform
            reinit=True)

        # Train the model for this fold and get the best validation f1
        best_val_f1 = train_model_with_hyperparams(model, model_name, train_loader, val_loader, optimizer, criterion, epochs = epochs, patience=patience, trial=trial)


        # Append fold validation accuracy
        f1_scores.append(best_val_f1)

        # Finish the Weights & Biases run
        wandb.finish()

    mean_f1 = np.mean(f1_scores)

    # Clean up CUDA + wandb
    import gc
    torch.cuda.empty_cache()
    gc.collect()

    if wandb.run is not None:
        wandb.finish()

    return mean_f1 # Return best mean validation f1

In [ ]:
n_trials = 3
pbar = tqdm(total=n_trials, desc="Optuna Trials")

def tqdm_callback(study, trial):
    pbar.update(1)

# Optimize - Run Optuna study
bertweet_study = optuna.create_study(direction="maximize") # Specifies that the goal of the optimization is to maximize the objective function (optimize mean f1)
bertweet_study.optimize(lambda trial: objective(trial, df_train, tokenizer, "vinai/bertweet-base", "TweetWithDateLocation", "label", device), n_trials=n_trials, callbacks=[tqdm_callback])

pbar.close()

# Get best trial info
best_trial = bertweet_study.best_trial
bertweet_best_params = bertweet_study.best_params


Optuna Trials:   0%|          | 0/5 [3:45:38<?, ?it/s]

Optuna Trials: 100%|██████████| 3/3 [2:27:02<00:00, 2940.77s/it]


In [ ]:
# Print the best hyperparameters
print("Best trial:")
print(f"  Validation F1-score: {best_trial.value}")
print("  Hyperparameters: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  Validation F1-score: 0.5991695538284437
  Hyperparameters: 
    learning_rate: 3.029705538459909e-05
    weight_decay: 8.155189459836697e-06
    patience: 5
    batch_size: 32
    epochs: 10
    num_layers: 1


#### Train again and test

Instead of saving the best model from a single fold during cross-validation, we need to retrain the model from scratch on the entire training set (excluding the test set), using the best hyperparameters selected by Optuna. This is applied in order to:
Maximizing Training Data: Retraining on the full dataset allowed the model to benefit from more examples than in any single fold, potentially improving generalization
Avoiding Fold Bias: Since no single fold sees the full training pool during cross-validation, retraining ensures the final model is not biased toward a specific validation split

In [12]:
def retrain_transformer_full(train_df, test_df, best_params, tokenizer, model_name, label_col, text_col, device=None):

    """
    Retrain the model on the full train_df using the best found hyperparameters form the Optuna study
    """

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare dataset and dataloader for full train
    train_dataset = TweetDataset(train_df, tokenizer, text_col=text_col, label_col=label_col, max_len=128)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)


    test_dataset = TweetDataset(test_df, tokenizer, text_col=text_col, label_col=label_col, max_len=128)
    test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False)

    # Initialize a fresh model for final training
    try:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)
    except:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, use_safetensors=True).to(device)

    #model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, use_safetensors=True).to(device)

    # Freeze & unfreeze layers according to best_params
    num_layers = best_params['num_layers']

    # Freezing logic for DeBERTa
    if hasattr(model, "deberta") and hasattr(model.deberta, "encoder"):
        # Freeze all DeBERTa encoder parameters
        for param in model.deberta.parameters():
            param.requires_grad = False

        # Unfreeze the last `num_layers` DeBERTa encoder layers
        encoder_layers = model.deberta.encoder.layer
        for layer in encoder_layers[-num_layers:]:
            for param in layer.parameters():
                param.requires_grad = True

    # Freezing logic for RoBERTa type models (e.g BERTweet)
    elif hasattr(model, "roberta") and hasattr(model.roberta, "encoder"):
        for param in model.roberta.parameters():
            param.requires_grad = False

        # Unfreeze the last `num_layers` of the encoder layers
        encoder_layers = model.roberta.encoder.layer
        for layer in encoder_layers[-num_layers:]:
            for param in layer.parameters():
                param.requires_grad = True

    # Always unfreeze classifier (for any model applied)
    for param in model.classifier.parameters():
        param.requires_grad = True

    optimizer = optim.Adam(model.parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'])
    criterion = nn.CrossEntropyLoss()

    # Start W&B logging for the final training
    run_name = f"Final_Train_{model_name.replace('/', '-')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    wandb.init(
        project=f"Final_{model_name.replace('/', '-')}_Retrain_{datetime.now().date().strftime('%Y%m%d')}",
        config=best_params,
        name=run_name,
        reinit=True,
    )

    # Retrain model on train_loader and evaluate on test_loader
    train_model_with_hyperparams(
        model,
        model_name=model_name,
        train_loader=train_loader,
        val_loader=test_loader,
        optimizer=optimizer,
        criterion=criterion,
        epochs=best_params['epochs'],
        patience=None,  # No early stopping
        trial=None      # Not an Optuna trial anymore
    )

    wandb.finish()

    return model

In [ ]:
# BERTweet tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

final_bertweet_model = retrain_transformer_full(
    train_df=df_train,
    test_df=df_test,
    best_params=bertweet_best_params,
    tokenizer=tokenizer,
    model_name='vinai/bertweet-base',
    label_col='label',
    text_col='TweetWithDateLocation',
    device=device
)

#### Evaluation Function

In [ ]:
def evaluate_model(model, test_loader, criterion, device=None):
    # Infer device from model if not provided
    if device is None:
        device = next(model.parameters()).device

    model.eval()
    all_labels, all_preds = [], []
    total_loss, total_samples = 0.0, 0

    start_time = time.time()

    with torch.no_grad():
        for batch in test_loader:
            # Move batch to the SAME device as the model
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            # label key can be 'labels' (PyTorch dataset) or 'label' (HF dataset); here it's 'labels'
            labels = (batch.get('labels') if 'labels' in batch else batch['label']).to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)

            total_loss += loss.item() * input_ids.size(0)
            total_samples += input_ids.size(0)

            preds = logits.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    end_time = time.time()
    inference_time = end_time - start_time
    inference_time_per_sample = inference_time / max(total_samples, 1)

    avg_loss = total_loss / max(total_samples, 1)
    accuracy = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    macro_precision = precision_score(all_labels, all_preds, average='macro')
    macro_recall = recall_score(all_labels, all_preds, average='macro')

    # Model size (MB)
    torch.save(model.state_dict(), "tmp_model_eval.pt")
    model_size_mb = os.path.getsize("tmp_model_eval.pt") / 1e6
    os.remove("tmp_model_eval.pt")

    # Parameter count
    param_count = sum(p.numel() for p in model.parameters())

    return {
        "Loss": avg_loss,
        "F1 Score (macro)": macro_f1,
        "Accuracy": accuracy,
        "Precision (macro)": macro_precision,
        "Recall (macro)": macro_recall,
        "Inference Time (sec)": inference_time,
        "Inference Time (sec/sample)": inference_time_per_sample,
        "Model Size (Mb)": model_size_mb,
        "Parameter Count": param_count
    }

In [ ]:
# Re-create test_dataset and test_loader outside
test_dataset = TweetDataset(df_test, tokenizer, text_col="TweetWithDateLocation", label_col="label", max_len=128)
test_loader = DataLoader(test_dataset, batch_size=bertweet_best_params['batch_size'], shuffle=False)

criterion = nn.CrossEntropyLoss()

metrics = evaluate_model(final_bertweet_model, test_loader, criterion, device)
for k, v in metrics.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

Loss: 1.0509
F1 Score (macro): 0.6111
Accuracy: 0.5964
Precision (macro): 0.6174
Recall (macro): 0.6069
Inference Time (sec): 7.4958
Inference Time (sec/sample): 0.0020
Model Size (Mb): 539.6985
Parameter Count: 134903813


### Hugging Face Fine-Tuning

#### Data Helper Functions

In [14]:
# Define Metrics Computation Function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    macro_f1 = f1_score(labels, preds, average='macro')
    macro_precision = precision_score(labels, preds, average='macro')
    macro_recall = recall_score(labels, preds, average='macro')
    accuracy = accuracy_score(labels, preds)
    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall
    }

In [15]:
 # Function to create a Hugging Face dataset compatible with Trainer
def make_hf_dataset(df, tokenizer, text_col, label_col, max_len, torch_format=False):
    """
    Convert a pandas DataFrame to a Hugging Face Dataset, tokenize, and optionally set torch format.
    Args:
        df: pandas DataFrame
        tokenizer: Hugging Face tokenizer
        text_col: str, name of text column
        label_col: str, name of label column
        max_len: int, max sequence length
        torch_format: bool, if True set format for PyTorch DataLoader (for evaluation)
    Returns:
        tokenized datasets.Dataset (optionally as torch tensors)
    """

    dataset = HFDataset.from_pandas(df)
    def preprocess(examples):
        return tokenizer(
            examples[text_col],
            truncation=True,
            padding="max_length",
            max_length=max_len,
        )
    dataset = dataset.map(preprocess, batched=True)
    if label_col != "label":
        dataset = dataset.rename_column(label_col, "label")
    if torch_format:
        dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    return dataset

In [23]:
def make_dual_hf_dataset(df, student_tok, teacher_tok, text_col, label_col, max_len):
    ds = HFDataset.from_pandas(df)

    def preprocess(examples):
        texts = examples[text_col]

        s_enc = student_tok(texts, truncation=True, padding="max_length", max_length=max_len)
        t_enc = teacher_tok(texts, truncation=True, padding="max_length", max_length=max_len)

        out = {
            "student_input_ids": s_enc["input_ids"],
            "student_attention_mask": s_enc["attention_mask"],
            "teacher_input_ids": t_enc["input_ids"],
            "teacher_attention_mask": t_enc["attention_mask"],
            "labels": examples[label_col],   # <-- create ONLY this label column
        }

        if "token_type_ids" in s_enc:
            out["student_token_type_ids"] = s_enc["token_type_ids"]
        if "token_type_ids" in t_enc:
            out["teacher_token_type_ids"] = t_enc["token_type_ids"]

        return out

    # Remove the original text + label columns so we don't keep a duplicate 'label'
    cols_to_remove = [c for c in ds.column_names if c in {text_col, label_col}]
    ds = ds.map(preprocess, batched=True, remove_columns=cols_to_remove)

    # Torch format: include only columns that exist
    torch_cols = [c for c in ds.column_names if c != "__index_level_0__"]
    ds.set_format(type="torch", columns=torch_cols)
    return ds

#### Training Scheme using HF API

In this training function we implement all the training needed in one function. There is the HP tuning with optuna and cross validation as well as full re-training on the full train data set with the best found HP from optuna.

In [24]:
def train_hf_model_with_cv_optuna(model_name, df_train, text_col, label_col, max_len=128, n_splits=3, n_trials=3, device=None, project_name=None):

    """
    Full pipeline: HP search with study-trial-fold logic, final retrain, returns best model and HPs.
    """

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False) # Load the model tokenizer

     # For Optuna: define the objective function for a single HP set (trial)
    def objective(trial):
        #  HP serach space
        learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
        weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-4)
        batch_size = trial.suggest_categorical("batch_size", [32, 64])
        num_epochs = trial.suggest_int("num_train_epochs", 10, 25, step=5)
        patience = trial.suggest_int("patience", 5, 7)
        num_layers = trial.suggest_int("num_layers", 1, 1)  # For freezing

        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        X = df_train[text_col].tolist()
        y = df_train[label_col].tolist()

        fold_f1s = []

        for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            # Prepare split
            X_train, X_val = [X[i] for i in train_idx], [X[i] for i in val_idx]
            y_train, y_val = [y[i] for i in train_idx], [y[i] for i in val_idx]

            train_fold_df = pd.DataFrame({text_col: X_train, label_col: y_train})
            val_fold_df = pd.DataFrame({text_col: X_val, label_col: y_val})

            train_dataset = make_hf_dataset(train_fold_df, tokenizer, text_col, label_col, max_len)
            val_dataset = make_hf_dataset(val_fold_df, tokenizer, text_col, label_col, max_len)

            # Model init with freezing logic
            def model_init():
                try:
                    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
                except:
                    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, use_safetensors=True)

                #model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, use_safetensors=True)

                # Freezing logic for DeBERTa
                if hasattr(model, "deberta") and hasattr(model.deberta, "encoder"):
                    # Freeze all DeBERTa encoder parameters
                    for param in model.deberta.parameters():
                        param.requires_grad = False

                    # Unfreeze the last `num_layers` DeBERTa encoder layers
                    encoder_layers = model.deberta.encoder.layer
                    for layer in encoder_layers[-num_layers:]:
                        for param in layer.parameters():
                            param.requires_grad = True

                # Freezing logic for RoBERTa type models (e.g BERTweet)
                elif hasattr(model, "roberta") and hasattr(model.roberta, "encoder"):
                    for param in model.roberta.parameters():
                        param.requires_grad = False

                    encoder_layers = model.roberta.encoder.layer
                    for layer in encoder_layers[-num_layers:]:
                        for param in layer.parameters():
                            param.requires_grad = True

                # Always unfreeze classifier (for any model applied)
                for param in model.classifier.parameters():
                    param.requires_grad = True

                return model.to(device)

            # Set up Trainer/Args
            training_args = TrainingArguments(
                output_dir=f"./results/{model_name.replace('/', '-')}-fold{fold+1}",    # Directory where model checkpoints, logs, and outputs will be saved
                report_to="wandb",                                                      # Weights & Biases logging
                run_name=f"{model_name}-CV-Fold{fold+1}",                               # Name of the run for logging and tracking in W&B
                eval_strategy="epoch",                                            # Run evaluation after each epoch
                save_strategy="epoch",                                                  # Save checkpoint after each epoch
                per_device_train_batch_size=batch_size,                                 # Batch size for training on each device (GPU/CPU)
                per_device_eval_batch_size=batch_size,                                  # Batch size for evaluation on each device (GPU/CPU)
                num_train_epochs=num_epochs,                                            # Number of epochs to train for
                learning_rate=learning_rate,
                weight_decay=weight_decay,
                logging_steps=50,                                                       # How often (in steps) to log metrics during training
                seed=42,                                                                # Random seed for reproducibility - Set for consistent result
                disable_tqdm=False,                                                     # False is default (have tqdm progress bars)
                load_best_model_at_end=True,                                            # Loads the best checkpoint according to metric_for_best_model at end of training
                metric_for_best_model="macro_f1",                                       # Which metric to use for "best model" selection
                greater_is_better=True,                                                 # Set True if higher is better for metric_for_best_model (e.g., F1)
                save_total_limit=1,                                                     # Max number of saved checkpoints. Older ones get deleted
            )

            # Start a new run for this trial/fold
            run_name = f"{model_name}-optuna-trial{trial.number}-fold{fold}"
            wandb.init(
                project=project_name,
                name=run_name,
                config={
                    "trial_number": trial.number,
                    "fold": fold,
                    "learning_rate": learning_rate,
                    "weight_decay": weight_decay,
                    "batch_size": batch_size,
                    "num_epochs": num_epochs,
                    "patience": patience,
                    "num_layers": num_layers
                },
                reinit=True,
            )

            callbacks = [EarlyStoppingCallback(early_stopping_patience=patience)]

            trainer = Trainer(
                model_init=model_init,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=val_dataset,
                compute_metrics=compute_metrics,
                callbacks=callbacks,
            )

            trainer.train()
            eval_metrics = trainer.evaluate()
            fold_f1s.append(eval_metrics['eval_macro_f1'])

            wandb.finish()

        mean_f1 = np.mean(fold_f1s)
        return mean_f1


    pbar = tqdm(total=n_trials, desc="Optuna Trials")

    def tqdm_callback(study, trial):
        pbar.update(1)

    # Run Optuna study (can add tqdm callback)
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials,callbacks=[tqdm_callback])

    pbar.close()

    best_params = study.best_trial.params
    print("Best HPs found:", best_params)

    # Final retrain on full training data
    full_train_ds = make_hf_dataset(df_train, tokenizer, text_col, label_col, max_len)

    def final_model_init():
        try:
            model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
        except:
            model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, use_safetensors=True)

        #model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, use_safetensors=True)

        # Freezing logic for DeBERTa
        if hasattr(model, "deberta") and hasattr(model.deberta, "encoder"):
            # Freeze all DeBERTa encoder parameters
            for param in model.deberta.parameters():
                param.requires_grad = False

            # Unfreeze the last `num_layers` DeBERTa encoder layers
            num_layers = best_params['num_layers']
            encoder_layers = model.deberta.encoder.layer
            for layer in encoder_layers[-num_layers:]:
                for param in layer.parameters():
                    param.requires_grad = True

        # Freezing logic for RoBERTa type models (e.g BERTweet)
        elif hasattr(model, "roberta") and hasattr(model.roberta, "encoder"):
            for param in model.roberta.parameters():
                param.requires_grad = False

            num_layers = best_params['num_layers']
            encoder_layers = model.roberta.encoder.layer
            for layer in encoder_layers[-num_layers:]:
                for param in layer.parameters():
                    param.requires_grad = True

        # Always unfreeze classifier (for any model applied)
        for param in model.classifier.parameters():
            param.requires_grad = True

        return model.to(device)

    # W&B logging ON for final training
    run_name = f"{model_name}-final-retrain"
    wandb.init(project=project_name, name=run_name, config=best_params, reinit=True)

    # Prepare TrainingArguments with best HPs
    final_training_args = TrainingArguments(
        output_dir=f"./results/{model_name.replace('/', '-')}-full-retrain",
        report_to="wandb",
        run_name=run_name,
        eval_strategy="no",  # No evaluation
        save_strategy="no",
        load_best_model_at_end=False,
        metric_for_best_model="macro_f1",
        greater_is_better=True,
        per_device_train_batch_size=best_params['batch_size'],
        num_train_epochs=best_params['num_train_epochs'],
        learning_rate=best_params['learning_rate'],
        weight_decay=best_params['weight_decay'],
        logging_steps=50,
        save_total_limit=1,
        seed=42
    )

    final_trainer = Trainer(
        model_init=final_model_init,
        args=final_training_args,
        train_dataset=full_train_ds,
        eval_dataset=None,  # No validation!
        compute_metrics=None,  # No eval during train
        callbacks=None,  # No patience in last full training - only needed in order to find best HP
    )

    # Train on full training data
    final_trainer.train()
    final_trainer.save_model(f"./{model_name.replace('/', '-')}_fine_tuned_hf_model") # Save full model

    # Save state_dict for PyTorch-based loading
    torch.save(final_trainer.model.state_dict(), f"{model_name.replace('/', '-')}_fine_tuned_hf_model.pt")

    wandb.finish()

    # Return final model and best params
    return final_trainer.model, best_params

#### Evaluation Function

In [25]:
def evaluate_hf_model_on_test(model, tokenizer, test_df, batch_size=32, text_col="TweetWithDateLocation", label_col="label", max_len=128, device=None):
    """
    Evaluates a Hugging Face model on the test set, reporting all key metrics:
    - Loss, F1 (macro), Accuracy, Precision, Recall, Inference Time (total, per sample),
    - Model Size (MB), Parameter Count
    """

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    model.eval() #  Set the model to evaluation mode

    # Prepare dataset and DataLoader
    test_dataset = make_hf_dataset(test_df, tokenizer, text_col, label_col, max_len, torch_format=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    all_labels, all_preds = [], []
    total_loss, total_samples = 0.0, 0

    start_time = time.time() # Start inference timer

    with torch.no_grad(): # Disable gradient computation for inference
        for batch in test_loader:
            # Move batch tensors to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Model forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = F.cross_entropy(logits, labels)

            # Accumulate loss and sample count
            total_loss += loss.item() * input_ids.size(0)
            total_samples += input_ids.size(0)

            # Collect predictions and true labels
            preds = logits.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    end_time = time.time() # End inference timer
    inference_time = end_time - start_time # Total inference time (all test samples)
    inference_time_per_sample = inference_time / total_samples

    # Compute performance metrics
    avg_loss = total_loss / total_samples
    accuracy = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    macro_precision = precision_score(all_labels, all_preds, average='macro')
    macro_recall = recall_score(all_labels, all_preds, average='macro')

    # Model size (MB)
    model_dir = "./tmp_eval_model"
    model.save_pretrained(model_dir)
    total_bytes = sum(
    os.path.getsize(os.path.join(model_dir, f))
    for f in os.listdir(model_dir)
    if os.path.isfile(os.path.join(model_dir, f))
    )
    model_size_mb = total_bytes / 1e6

    shutil.rmtree(model_dir)  # Clean up

    # Parameter count
    param_count = sum(p.numel() for p in model.parameters())

    # Return as dict
    return {
        "Loss": avg_loss,
        "F1 Score (macro)": macro_f1,
        "Accuracy": accuracy,
        "Precision (macro)": macro_precision,
        "Recall (macro)": macro_recall,
        "Inference Time (sec)": inference_time,
        "Inference Time (sec/sample)": inference_time_per_sample,
        "Model Size (Mb)": model_size_mb,
        "Parameter Count": param_count
    }

Call the training and evaluation function on each of the two selected models

In [ ]:
# For BERTweet

# Train and returns the fully fine-tuned model in-memory (bertweet_model), and the best hyperparameters found (bertweet_best_params)
# The trained model is also saved to ./vinai-bertweet-base-hf-final-best
bertweet_model, bertweet_best_params = train_hf_model_with_cv_optuna(
    model_name="vinai/bertweet-base",
    df_train=df_train,
    text_col="TweetWithDateLocation",
    label_col="label",
    max_len=128,
    n_splits=3,
    n_trials=3,
    device=device,
    project_name="BERTweet-Sentiment-CV"
)

# Evaluate BERTweet model using the saved fully trained model from the disk

# Reload the best model from disk
try:
    final_bertweet_model = AutoModelForSequenceClassification.from_pretrained(
    "./vinai-bertweet-base_fine_tuned_hf_model").to(device)
except:
    final_bertweet_model = AutoModelForSequenceClassification.from_pretrained(
    "./vinai-bertweet-base_fine_tuned_hf_model", use_safetensors=True).to(device)

#final_bertweet_model = AutoModelForSequenceClassification.from_pretrained(
#    "./vinai-bertweet-base_fine_tuned_hf_model", use_safetensors=True
#).to(device)

bertweet_metrics = evaluate_hf_model_on_test(
    final_bertweet_model,
    AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False),
    df_test,
    batch_size=bertweet_best_params['batch_size'],
    text_col="TweetWithDateLocation",
    label_col="label",
    max_len=128,
    device=device
)

print("BERTweet Test Results:")
for k, v in bertweet_metrics.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")


Optuna Trials:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/27434 [00:00<?, ? examples/s]

Map:   0%|          | 0/13718 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Macro F1,Macro Precision,Macro Recall
1,1.302800,1.301235,0.429727,0.431340,0.457214,0.423672
2,1.267700,1.234502,0.466540,0.478993,0.482459,0.476180
3,1.213600,1.197807,0.479224,0.491173,0.497075,0.490704
4,1.185400,1.158046,0.500948,0.514344,0.516564,0.514161
5,1.122700,1.138525,0.509477,0.523169,0.518994,0.529957
6,1.139300,1.119181,0.521505,0.536137,0.541804,0.531578
7,1.146800,1.108770,0.525077,0.539908,0.542032,0.539571
8,1.102200,1.108351,0.529159,0.544495,0.542445,0.550882
9,1.084700,1.094532,0.534918,0.550005,0.546961,0.555278
10,1.079400,1.086185,0.537177,0.551602,0.559057,0.546976


Map:   0%|          | 0/27435 [00:00<?, ? examples/s]

...

Best HPs found: {'learning_rate': 0.000488657939143141, 'weight_decay': 1.3929919520517762e-06, 'batch_size': 32, 'num_train_epochs': 20, 'patience': 6, 'num_layers': 1}


Map:   0%|          | 0/41152 [00:00<?, ? examples/s]

{'loss': 1.5371, 'grad_norm': 1.7142730951309204, 'learning_rate': 0.00048772698117198953, 'epoch': 0.038880248833592534}
{'loss': 1.3676, 'grad_norm': 1.1776220798492432, 'learning_rate': 0.00048677702405856973, 'epoch': 0.07776049766718507}
{'loss': 1.2851, 'grad_norm': 2.8596901893615723, 'learning_rate': 0.00048582706694515, 'epoch': 0.1166407465007776}
{'loss': 1.2355, 'grad_norm': 1.9892528057098389, 'learning_rate': 0.0004848771098317302, 'epoch': 0.15552099533437014}
{'loss': 1.251, 'grad_norm': 2.6065616607666016, 'learning_rate': 0.00048392715271831037, 'epoch': 0.19440124416796267}
{'loss': 1.1731, 'grad_norm': 1.2463252544403076, 'learning_rate': 0.00048297719560489057, 'epoch': 0.2332814930015552}
{'loss': 1.1808, 'grad_norm': 2.8448750972747803, 'learning_rate': 0.00048202723849147076, 'epoch': 0.27216174183514774}
{'loss': 1.1375, 'grad_norm': 2.1501920223236084, 'learning_rate': 0.00048107728137805096, 'epoch': 0.3110419906687403}
{'loss': 1.1898, 'grad_norm': 1.9919151

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

BERTweet Test Results:
Loss: 1.1460
F1 Score (macro): 0.6261
Accuracy: 0.6111
Precision (macro): 0.6275
Recall (macro): 0.6248
Inference Time (sec): 5.8868
Inference Time (sec/sample): 0.0015
Model Size (Mb): 539.6404
Parameter Count: 134903813


## Fine-tuning DeBERTa

DeBERTa (Decoding-enhanced BERT with Disentangled Attention) is an advanced Transformer model that has achieved state-of-the-art performance on multiple NLP benchmarks, including sentiment classification. Developed by Microsoft (He et al., 2021), DeBERTa introduces innovations like disentangled attention (separating content and position embeddings) and an enhanced decoder, leading to superior language understanding. In sentiment analysis research, DeBERTa’s strength is evident in both traditional and social media domains. For example, the DeBERTa-large model reached 96.8% accuracy on the SST-2 sentiment benchmark (Stanford movie reviews).Moreover, researchers have applied DeBERTa to Twitter data: Assiri et al. (2024) (https://www.techscience.com/cmc/v79n3/57133#:~:text=Recurrent%20Unit%20,the%20performance%20of%20new%20techniques) combined DeBERTa with a GRU network for tweet sentiment analysis and reported an exceptionally high F1 score (~97%) on a large Twitter sentiment dataset. These results illustrate DeBERTa’s ability to capture sentiment cues with high precision. DeBERTa is readily available on Hugging Face and fully compatible with PyTorch, making it convenient for fine-tuning to our social media datasets. In summary, DeBERTa is a top-performing Transformer model for text classification tasks; its peer-reviewed evaluations show superior sentiment analysis performance, and its models can be directly used via Hugging Face for sentiment classification on platforms like Twitter.

In [26]:
# Load the DeBERTa base model
try:
    model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=5)
except:
    model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=5, use_safetensors=True)

#model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=5, use_safetensors=True)

# Print the model architecture
print(model)

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): Dropout(p=0

### PyTorch Fine-Tuning + Evaluation

In [ ]:
# Set model name for DeBERTa
model_name = "microsoft/deberta-base"

# Tokenizer for DeBERTa
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Run Optuna study
n_trials = 3
pbar = tqdm(total=n_trials, desc="Optuna Trials")

def tqdm_callback(study, trial):
    pbar.update(1)

deberta_study = optuna.create_study(direction="maximize")
deberta_study.optimize(
    lambda trial: objective(trial, df_train, tokenizer, model_name, "TweetWithDateLocation", "label", device),
    n_trials=n_trials, callbacks=[tqdm_callback]
)
pbar.close()

best_trial = deberta_study.best_trial
deberta_best_params = deberta_study.best_params

# Print the best hyperparameters
print("Best trial:")
print(f"  Validation F1-score: {best_trial.value}")
print("  Hyperparameters: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

# Retrain final model on full data
final_deberta_model = retrain_transformer_full(
    train_df=df_train,
    test_df=df_test,
    best_params=deberta_best_params,
    tokenizer=tokenizer,
    model_name=model_name,
    label_col='label',
    text_col='TweetWithDateLocation',
    device=device
)

# Re-create test_dataset and test_loader
test_dataset = TweetDataset(df_test, tokenizer, text_col="TweetWithDateLocation", label_col="label", max_len=128)
test_loader = DataLoader(test_dataset, batch_size=deberta_best_params['batch_size'], shuffle=False)

criterion = nn.CrossEntropyLoss()

# Evalute fully retrained model
metrics = evaluate_model(final_deberta_model, test_loader, criterion, device)
for k, v in metrics.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")


Optuna Trials:   0%|          | 0/5 [00:33<?, ?it/s]

Optuna Trials: 100%|██████████| 3/3 [3:20:33<00:00, 4011.14s/it]


Best trial:
  Validation F1-score: 0.6219160488365016
  Hyperparameters: 
    learning_rate: 1.2742033340580216e-05
    weight_decay: 8.329678149975656e-06
    patience: 5
    batch_size: 32
    epochs: 15
    num_layers: 1
Loss: 0.9614
F1 Score (macro): 0.6207
Accuracy: 0.6051
Precision (macro): 0.6205
Recall (macro): 0.6250
Inference Time (sec): 9.7030
Inference Time (sec/sample): 0.0026
Model Size (Mb): 556.8687
Parameter Count: 139196165


### Hugging Face Fine-Tuning + Evaluation

In [ ]:
# For DeBERTa

# Train and return the fully fine-tuned model in-memory (deberta_model), and the best hyperparameters found (deberta_best_params)
# The trained model is also saved to ./microsoft-deberta-base-hf-final-best
deberta_model, deberta_best_params = train_hf_model_with_cv_optuna(
    model_name="microsoft/deberta-base",
    df_train=df_train,
    text_col="TweetWithDateLocation",
    label_col="label",
    max_len=128,
    n_splits=3,
    n_trials=3,
    device=device,
    project_name="DeBERTa-Sentiment-CV"
)

# Evaluate DeBERTa model using the saved fully trained model from the disk

# Reload the best model from disk
try:
    final_deberta_model = AutoModelForSequenceClassification.from_pretrained(
    "./microsoft-deberta-base_fine_tuned_hf_model").to(device)
except:
    final_deberta_model = AutoModelForSequenceClassification.from_pretrained(
    "./microsoft-deberta-base_fine_tuned_hf_model", use_safetensors=True).to(device)


deberta_metrics = evaluate_hf_model_on_test(
    final_deberta_model,
    AutoTokenizer.from_pretrained("microsoft/deberta-base", use_fast=False),
    df_test,
    batch_size=deberta_best_params['batch_size'],
    text_col="TweetWithDateLocation",
    label_col="label",
    max_len=128,
    device=device
)

print("DeBERTa Test Results:")
for k, v in deberta_metrics.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Optuna Trials:   0%|          | 0/3 [00:00<?, ?it/s]

...

Best HPs found: {'learning_rate': 0.0002983293259160811, 'weight_decay': 3.1187918540503307e-06, 'batch_size': 64, 'num_train_epochs': 15, 'patience': 6, 'num_layers': 1}


Map:   0%|          | 0/41152 [00:00<?, ? examples/s]

{'loss': 1.5637, 'grad_norm': 0.9810400605201721, 'learning_rate': 0.000296813707775087, 'epoch': 0.07776049766718507}
{'loss': 1.332, 'grad_norm': 1.7525864839553833, 'learning_rate': 0.0002952671586516236, 'epoch': 0.15552099533437014}
{'loss': 1.2288, 'grad_norm': 1.8302069902420044, 'learning_rate': 0.0002937206095281603, 'epoch': 0.2332814930015552}
{'loss': 1.17, 'grad_norm': 2.1091220378875732, 'learning_rate': 0.0002921740604046969, 'epoch': 0.3110419906687403}
{'loss': 1.1347, 'grad_norm': 1.3891545534133911, 'learning_rate': 0.0002906275112812336, 'epoch': 0.38880248833592534}
{'loss': 1.1125, 'grad_norm': 2.357174873352051, 'learning_rate': 0.0002890809621577702, 'epoch': 0.4665629860031104}
{'loss': 1.0977, 'grad_norm': 3.5730929374694824, 'learning_rate': 0.0002875344130343069, 'epoch': 0.5443234836702955}
{'loss': 1.066, 'grad_norm': 1.4995447397232056, 'learning_rate': 0.0002859878639108435, 'epoch': 0.6220839813374806}
{'loss': 1.0452, 'grad_norm': 2.005154848098755, 'l

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

DeBERTa Test Results:
Loss: 0.9803
F1 Score (macro): 0.6421
Accuracy: 0.6280
Precision (macro): 0.6458
Recall (macro): 0.6393
Inference Time (sec): 7.2638
Inference Time (sec/sample): 0.0019
Model Size (Mb): 556.8098
Parameter Count: 139196165


## Compression Techniques

### Quantization

In [ ]:
def quantize_and_evaluate(model_name, state_dict_path, quantized_path, tokenizer,
                          df_test, text_col, label_col, max_len, batch_size,
                          device, criterion):
    # Force CPU for quantization & eval (quantized ops are CPU-only)
    cpu_device = torch.device('cpu')

    # Load model architecture on CPU
    try:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(cpu_device)
    except:
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=5, use_safetensors=True
        ).to(cpu_device)

    # Load fine-tuned weights on CPU
    state_dict = torch.load(state_dict_path, map_location=cpu_device)
    model.load_state_dict(state_dict)
    model.eval()

    # Dynamic quantization (Linear layers) on CPU
    with torch.no_grad():
        quantized_model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
    quantized_model.eval()
    quantized_model.to(cpu_device)

    # Save quantized model weights
    torch.save(quantized_model.state_dict(), quantized_path)
    print(f"Quantized model saved as {quantized_path}")

    # Build test loader
    test_dataset = TweetDataset(df_test, tokenizer, text_col=text_col, label_col=label_col, max_len=max_len)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Evaluate ON CPU (device is ignored; we pass None so eval infers from model)
    metrics = evaluate_model(quantized_model, test_loader, criterion, device=None)
    print(f"Evaluation Metrics for {model_name.replace('/', '-')} Quantized Model:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

    return quantized_model, metrics

In [ ]:
from transformers import AutoTokenizer

model_configs_quantization = [
    {
        "name": "Fine-Tuned BERTweet PyTorch",
        "model_name": "vinai/bertweet-base",
        "state_dict_path": "vinai-bertweet-base_fine_tuned_pytorch_model.pt",
        "quantized_path": "vinai-bertweet-base_fine_tuned_pytorch_model_quantized.pt",
        "tokenizer": AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=True),
    },
    {
        "name": "Fine-Tuned DeBERTa PyTorch",
        "model_name": "microsoft/deberta-base",  # match the state dict
        "state_dict_path": "microsoft-deberta-base_fine_tuned_pytorch_model.pt",
        "quantized_path": "microsoft-deberta-base_fine_tuned_pytorch_model_quantized.pt",
        "tokenizer": AutoTokenizer.from_pretrained("microsoft/deberta-base", use_fast=True),
    },
    {
        "name": "Fine-Tuned BERTweet Trainer",
        "model_name": "vinai/bertweet-base",
        "state_dict_path": "vinai-bertweet-base_fine_tuned_hf_model.pt",
        "quantized_path": "vinai-bertweet-base_fine_tuned_hf_model_quantized.pt",
        "tokenizer": AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=True),
    },
    {
        "name": "Fine-Tuned DeBERTa Trainer",
        "model_name": "microsoft/deberta-base",  # keep consistent
        "state_dict_path": "microsoft-deberta-base_fine_tuned_hf_model.pt",
        "quantized_path": "microsoft-deberta-base_fine_tuned_hf_model_quantized.pt",
        "tokenizer": AutoTokenizer.from_pretrained("microsoft/deberta-base", use_fast=True),
    },
]

# Run quantization + evaluation for each model and store results
all_quantization_metrics = []

for config in model_configs_quantization:
    print(f"\nQuantizing and evaluating: {config['name']}")

    quantized_model, metrics = quantize_and_evaluate(
        model_name=config["model_name"],
        state_dict_path=config["state_dict_path"],
        quantized_path=config["quantized_path"],
        tokenizer=config["tokenizer"],
        df_test=df_test,
        text_col="TweetWithDateLocation",
        label_col="label",
        max_len=128,
        batch_size=32,
        device=device,
        criterion=nn.CrossEntropyLoss(),
    )


    # Add model name to metrics
    metrics["Model"] = config["name"]
    all_quantization_metrics.append(metrics)

# Create DataFrame with results
df_quantized_metrics = pd.DataFrame(all_quantization_metrics)

# Reorder columns to place 'Model' first
cols = ["Model"] + [col for col in df_quantized_metrics.columns if col != "Model"]
df_quantized_metrics = df_quantized_metrics[cols]

# Display results
display(df_quantized_metrics)


Quantizing and evaluating: Fine-Tuned BERTweet PyTorch
Quantized model saved as vinai-bertweet-base_fine_tuned_pytorch_model_quantized.pt
Evaluation Metrics for vinai-bertweet-base Quantized Model:
Loss: 1.5785
F1 Score (macro): 0.1664
Accuracy: 0.3136
Precision (macro): 0.6163
Recall (macro): 0.2388
Inference Time (sec): 111.2537
Inference Time (sec/sample): 0.0293
Model Size (Mb): 283.1712
Parameter Count: 49291776

Quantizing and evaluating: Fine-Tuned DeBERTa PyTorch


pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

Quantized model saved as microsoft-deberta-base_fine_tuned_pytorch_model_quantized.pt


model.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

Evaluation Metrics for microsoft-deberta-base Quantized Model:
Loss: 1.5297
F1 Score (macro): 0.2514
Accuracy: 0.3133
Precision (macro): 0.4802
Recall (macro): 0.2913
Inference Time (sec): 175.0920
Inference Time (sec/sample): 0.0461
Model Size (Mb): 257.8754
Parameter Count: 39446784

Quantizing and evaluating: Fine-Tuned BERTweet Trainer
Quantized model saved as vinai-bertweet-base_fine_tuned_hf_model_quantized.pt
Evaluation Metrics for vinai-bertweet-base Quantized Model:
Loss: 2.0628
F1 Score (macro): 0.2179
Accuracy: 0.2809
Precision (macro): 0.3152
Recall (macro): 0.2909
Inference Time (sec): 81.7124
Inference Time (sec/sample): 0.0215
Model Size (Mb): 283.1712
Parameter Count: 49291776

Quantizing and evaluating: Fine-Tuned DeBERTa Trainer
Quantized model saved as microsoft-deberta-base_fine_tuned_hf_model_quantized.pt
Evaluation Metrics for microsoft-deberta-base Quantized Model:
Loss: 1.9091
F1 Score (macro): 0.3920
Accuracy: 0.4018
Precision (macro): 0.4556
Recall (macro): 0.

,Model,Loss,F1 Score (macro),Accuracy,Precision (macro),Recall (macro),Inference Time (sec),Inference Time (sec/sample),Model Size (Mb),Parameter Count
0,Fine-Tuned BERTweet PyTorch,1.578507,0.166389,0.313586,0.616260,0.238794,111.253694,0.029293,283.171231,49291776
1,Fine-Tuned DeBERTa PyTorch,1.529704,0.251361,0.313323,0.480186,0.291341,175.091951,0.046101,257.875418,39446784
2,Fine-Tuned BERTweet Trainer,2.062798,0.217926,0.280937,0.315240,0.290851,81.712414,0.021515,283.171231,49291776
3,Fine-Tuned DeBERTa Trainer,1.909088,0.391975,0.401790,0.455613,0.416467,182.651803,0.048092,257.875418,39446784


### Pruning

In [ ]:
def prune_and_evaluate(model_name, state_dict_path, pruned_path, tokenizer, df_test, text_col, label_col, max_len, batch_size, device, criterion,
                       amount=0.3,  # Fraction of weights to prune in each linear layer
                       prune_type='l1_unstructured', # Or 'random_unstructured'
                       ):

    # Load model architecture
    try:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)
    except:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, use_safetensors=True).to(device)

    #model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, use_safetensors=True).to(device)

    # Load weights of fine tuned model
    model.load_state_dict(torch.load(state_dict_path, map_location=device))
    model.eval()

    # Gather all linear layer weights for global pruning
    parameters_to_prune = []
    for module in model.modules():
        if isinstance(module, nn.Linear):
            parameters_to_prune.append((module, 'weight'))

    # Apply global unstructured pruning across all linear layers
    if prune_type == 'l1_unstructured':
        prune.global_unstructured(
            parameters_to_prune,
            pruning_method=prune.L1Unstructured,
            amount=amount,
        )
    elif prune_type == 'random_unstructured':
        prune.global_unstructured(
            parameters_to_prune,
            pruning_method=prune.RandomUnstructured,
            amount=amount,
        )
    else:
        raise ValueError("Unsupported prune_type")

    # Remove pruning reparametrization (make zeros permanent)
    for module, name in parameters_to_prune:
        prune.remove(module, name)

    # Save pruned model
    torch.save(model.state_dict(), pruned_path)
    print(f"Pruned model saved as {pruned_path}")

    # Evaluate pruned model metrics
    test_dataset = TweetDataset(df_test, tokenizer, text_col=text_col, label_col=label_col, max_len=max_len)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    metrics = evaluate_model(model, test_loader, criterion, device)
    print(f"Evaluation Metrics for {model_name.replace('/', '-')} Pruned Model:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

    return model, metrics

In [ ]:
# Define config list
model_configs_pruning = [
    {
        "name": "Fine-Tuned BERTweet PyTorch",
        "model_name": "vinai/bertweet-base",
        "state_dict_path": "vinai-bertweet-base_fine_tuned_pytorch_model.pt",
        "pruned_path": "vinai-bertweet-base_fine_tuned_pytorch_model_pruned.pt",
        "tokenizer": AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=True),
    },
    {
        "name": "Fine-Tuned DeBERTa PyTorch",
        "model_name": "microsoft/deberta-base",  # match model to saved state dict
        "state_dict_path": "microsoft-deberta-base_fine_tuned_pytorch_model.pt",
        "pruned_path": "microsoft-deberta-base_fine_tuned_pytorch_model_pruned.pt",
        "tokenizer": AutoTokenizer.from_pretrained("microsoft/deberta-base", use_fast=True),
    },
    {
        "name": "Fine-Tuned BERTweet Trainer",
        "model_name": "vinai/bertweet-base",
        "state_dict_path": "vinai-bertweet-base_fine_tuned_hf_model.pt",
        "pruned_path": "vinai-bertweet-base_fine_tuned_hf_model_pruned.pt",
        "tokenizer": AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=True),
    },
    {
        "name": "Fine-Tuned DeBERTa Trainer",
        "model_name": "microsoft/deberta-base",  # match model to saved state dict
        "state_dict_path": "microsoft-deberta-base_fine_tuned_hf_model.pt",
        "pruned_path": "microsoft-deberta-base_fine_tuned_hf_model_pruned.pt",
        "tokenizer": AutoTokenizer.from_pretrained("microsoft/deberta-base", use_fast=True),
    },
]

# Run pruning + evaluation for each model and store results
all_pruned_metrics = []

for config in model_configs_pruning:
    print(f"\nPruning and evaluating: {config['name']}")

    pruned_model, metrics = prune_and_evaluate(
        model_name=config["model_name"],
        state_dict_path=config["state_dict_path"],
        pruned_path=config["pruned_path"],
        tokenizer=config["tokenizer"],
        df_test=df_test,
        text_col="TweetWithDateLocation",
        label_col="label",
        max_len=128,
        batch_size=32,
        device=device,
        criterion=nn.CrossEntropyLoss(),
        amount=0.3,  # 30% global sparsity
        prune_type='l1_unstructured' # 'l1_unstructured' or 'random_unstructured'
    )

    # Add model name to metrics
    metrics["Model"] = config["name"]
    all_pruned_metrics.append(metrics)

# Create DataFrame with results
df_pruned_metrics = pd.DataFrame(all_pruned_metrics)

# Reorder columns to place 'Model' first
cols = ["Model"] + [col for col in df_pruned_metrics.columns if col != "Model"]
df_pruned_metrics = df_pruned_metrics[cols]

# Display results
display(df_pruned_metrics)


Pruning and evaluating: Fine-Tuned BERTweet PyTorch
Pruned model saved as vinai-bertweet-base_fine_tuned_pytorch_model_pruned.pt
Evaluation Metrics for vinai-bertweet-base Pruned Model:
Loss: 1.0499
F1 Score (macro): 0.5791
Accuracy: 0.5714
Precision (macro): 0.5973
Recall (macro): 0.5818
Inference Time (sec): 8.5880
Inference Time (sec/sample): 0.0023
Model Size (Mb): 539.6995
Parameter Count: 134903813

Pruning and evaluating: Fine-Tuned DeBERTa PyTorch
Pruned model saved as microsoft-deberta-base_fine_tuned_pytorch_model_pruned.pt
Evaluation Metrics for microsoft-deberta-base Pruned Model:
Loss: 1.0263
F1 Score (macro): 0.5726
Accuracy: 0.5569
Precision (macro): 0.5803
Recall (macro): 0.5880
Inference Time (sec): 9.1112
Inference Time (sec/sample): 0.0024
Model Size (Mb): 556.8697
Parameter Count: 139196165

Pruning and evaluating: Fine-Tuned BERTweet Trainer
Pruned model saved as vinai-bertweet-base_fine_tuned_hf_model_pruned.pt
Evaluation Metrics for vinai-bertweet-base Pruned Mo

,Model,Loss,F1 Score (macro),Accuracy,Precision (macro),Recall (macro),Inference Time (sec),Inference Time (sec/sample),Model Size (Mb),Parameter Count
0,Fine-Tuned BERTweet PyTorch,1.049923,0.579104,0.571353,0.597312,0.581762,8.587952,0.002261,539.699515,134903813
1,Fine-Tuned DeBERTa PyTorch,1.026346,0.572582,0.556872,0.580270,0.588028,9.111203,0.002399,556.869686,139196165
2,Fine-Tuned BERTweet Trainer,1.116392,0.592346,0.590837,0.633732,0.593259,8.246823,0.002171,539.699515,134903813
3,Fine-Tuned DeBERTa Trainer,0.978490,0.610930,0.596103,0.613976,0.619502,8.793466,0.002315,556.869686,139196165


### Knowledge Distillation

In [36]:
from transformers import TrainerCallback
import pandas as pd

class CollectLogsCallback(TrainerCallback):
    def __init__(self):
        self.history = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            self.history.append({**logs, "epoch": state.epoch})

    def to_df(self):
        return pd.DataFrame(self.history)

In [49]:
import os, re, datetime as dt
import pandas as pd
import torch
import torch.nn.functional as F
import wandb

from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
from transformers.trainer_callback import PrinterCallback, TrainerCallback

# ------- W&B helpers (project naming + quiet logs to DF) -------
def set_wandb_project(teacher_model_name: str, student_model_name: str, prefix: str = "KD") -> str:
    t = teacher_model_name.split("/")[-1]
    s = student_model_name.split("/")[-1]
    today = dt.datetime.now().strftime("%Y%m%d")
    project = f"{prefix}-{t}-to-{s}-{today}"
    os.environ["WANDB_PROJECT"] = project            # HF default override
    os.environ["WANDB_RUN_GROUP"] = project          # group runs nicely
    os.environ["WANDB_SILENT"] = "true"              # reduce console noise
    return project

class CollectLogsCallback(TrainerCallback):
    def __init__(self):
        self.rows = []
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            row = {"step": state.global_step, "epoch": state.epoch}
            row.update(logs)
            self.rows.append(row)
    def to_df(self) -> pd.DataFrame:
        return pd.DataFrame(self.rows)

# ----------------- KD Trainer -----------------
class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, temperature=2.0, alpha=0.5, **kwargs):
        super().__init__(*args, **kwargs)
        assert teacher_model is not None, "teacher_model must be provided"
        self.teacher = teacher_model.eval()
        self.temperature = float(temperature)
        self.alpha = float(alpha)
        self.teacher.to(next(self.model.parameters()).device)

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs["labels"] if "labels" in inputs else inputs["label"]

        s_inputs = {
            "input_ids": inputs["student_input_ids"],
            "attention_mask": inputs["student_attention_mask"],
        }
        if "student_token_type_ids" in inputs:
            s_inputs["token_type_ids"] = inputs["student_token_type_ids"]

        t_inputs = {
            "input_ids": inputs["teacher_input_ids"],
            "attention_mask": inputs["teacher_attention_mask"],
        }
        if "teacher_token_type_ids" in inputs:
            t_inputs["token_type_ids"] = inputs["teacher_token_type_ids"]

        outputs_student = model(**s_inputs)
        with torch.inference_mode():
            outputs_teacher = self.teacher(**t_inputs)

        loss_ce = F.cross_entropy(outputs_student.logits, labels)
        T = self.temperature
        loss_kl = F.kl_div(
            F.log_softmax(outputs_student.logits / T, dim=-1),
            F.softmax(outputs_teacher.logits / T, dim=-1),
            reduction="batchmean",
        ) * (T ** 2)

        loss = self.alpha * loss_ce + (1.0 - self.alpha) * loss_kl
        return (loss, outputs_student) if return_outputs else loss


def distill_with_optuna_and_evaluation(
    teacher_model_name, teacher_state_dict_path, student_model_name,
    tokenizer_student, tokenizer_teacher,
    df_train, df_test, text_col, label_col, max_len,
    n_trials=3, n_splits=3, device=None, output_dir=".", distilled_from=None
):
    import optuna
    from sklearn.model_selection import StratifiedKFold

    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
    distilled_from = distilled_from or teacher_model_name

    # W&B project per teacher/student/date
    project = set_wandb_project(teacher_model_name, student_model_name, prefix="KD")

    # Load teacher + weights
    try:
        teacher = AutoModelForSequenceClassification.from_pretrained(teacher_model_name, num_labels=5).to(device)
    except:
        teacher = AutoModelForSequenceClassification.from_pretrained(
            teacher_model_name, num_labels=5, use_safetensors=True
        ).to(device)
    teacher.load_state_dict(torch.load(teacher_state_dict_path, map_location=device))
    teacher.eval()

    X = df_train[text_col].tolist()
    y = df_train[label_col].tolist()

    def kd_objective(trial):
        lr = trial.suggest_loguniform("learning_rate", 1e-5, 5e-4)
        wd = trial.suggest_loguniform("weight_decay", 1e-6, 1e-4)
        bs = trial.suggest_categorical("batch_size", [16, 32])
        epochs = trial.suggest_int("epochs", 3, 6)
        T = trial.suggest_float("temperature", 1.5, 4.0)
        alpha = trial.suggest_float("alpha", 0.2, 0.8)

        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        fold_f1 = []

        for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y)):
            tr_df = pd.DataFrame({text_col: [X[i] for i in tr_idx], label_col: [y[i] for i in tr_idx]})
            va_df = pd.DataFrame({text_col: [X[i] for i in va_idx], label_col: [y[i] for i in va_idx]})

            train_ds = make_dual_hf_dataset(tr_df, tokenizer_student, tokenizer_teacher, text_col, label_col, max_len)
            val_ds   = make_dual_hf_dataset(va_df, tokenizer_student, tokenizer_teacher, text_col, label_col, max_len)

            try:
                student = AutoModelForSequenceClassification.from_pretrained(student_model_name, num_labels=5)
            except:
                student = AutoModelForSequenceClassification.from_pretrained(
                    student_model_name, num_labels=5, use_safetensors=True
                )
            student.to(device)

            args = TrainingArguments(
                output_dir=f"./tmp_kd_student_fold_{fold}",
                learning_rate=lr,
                weight_decay=wd,
                per_device_train_batch_size=bs,
                per_device_eval_batch_size=bs,
                num_train_epochs=epochs,
                eval_strategy="epoch",
                save_strategy="no",
                logging_strategy="epoch",     # log once/epoch
                disable_tqdm=True,            # hide progress bars
                load_best_model_at_end=False,
                report_to="wandb",
                run_name=f"KD-optuna-fold{fold}",
                fp16=torch.cuda.is_available(),
                remove_unused_columns=False,
                log_level="error",
                log_on_each_node=False,
            )

            # Start an explicit W&B run so project is NOT "huggingface"
            wandb.init(project=project, name=f"KD-optuna-fold{fold}",
                       group=project,
                       config={"lr": lr, "wd": wd, "bs": bs, "epochs": epochs, "T": T, "alpha": alpha},
                       reinit=True)

            trainer = DistillationTrainer(
                model=student,
                teacher_model=teacher,
                args=args,
                train_dataset=train_ds,
                eval_dataset=val_ds,
                temperature=T,
                alpha=alpha,
                compute_metrics=compute_metrics,
            )
            trainer.remove_callback(PrinterCallback)  # stop dict spam

            trainer.train()
            metrics = trainer.evaluate()
            fold_f1.append(metrics["eval_macro_f1"])

            wandb.finish()
            del student
            torch.cuda.empty_cache()

        return float(pd.Series(fold_f1).mean())

    # Optuna search
    study = optuna.create_study(direction="maximize")
    study.optimize(kd_objective, n_trials=n_trials)
    best = study.best_params

    # Final train on full train set
    train_full = make_dual_hf_dataset(df_train, tokenizer_student, tokenizer_teacher, text_col, label_col, max_len)
    test_ds    = make_dual_hf_dataset(df_test,  tokenizer_student, tokenizer_teacher, text_col, label_col, max_len)

    try:
        student_final = AutoModelForSequenceClassification.from_pretrained(student_model_name, num_labels=5)
    except:
        student_final = AutoModelForSequenceClassification.from_pretrained(
            student_model_name, num_labels=5, use_safetensors=True
        )
    student_final.to(device)

    final_args = TrainingArguments(
        output_dir="./final_kd_student",
        learning_rate=best["learning_rate"],
        weight_decay=best["weight_decay"],
        per_device_train_batch_size=best["batch_size"],
        per_device_eval_batch_size=best["batch_size"],
        num_train_epochs=max(1, int(best["epochs"])),
        eval_strategy="epoch",
        save_strategy="no",
        logging_strategy="epoch",
        disable_tqdm=True,
        report_to="wandb",
        run_name=f"KD-final-{student_model_name.split('/')[-1]}",
        fp16=torch.cuda.is_available(),
        remove_unused_columns=False,
        log_level="error",
        log_on_each_node=False,
    )

    log_cb = CollectLogsCallback()
    wandb.init(project=project, name=f"KD-final-{student_model_name.split('/')[-1]}",
               group=project, config=best, reinit=True)

    trainer = DistillationTrainer(
        model=student_final,
        teacher_model=teacher,
        args=final_args,
        train_dataset=train_full,
        eval_dataset=test_ds,
        temperature=best["temperature"],
        alpha=best["alpha"],
        compute_metrics=compute_metrics,
        callbacks=[log_cb],
    )
    trainer.remove_callback(PrinterCallback)  # stop dict spam
    trainer.train()
    wandb.finish()

    # Display compact table instead of dict prints
    logs_df = log_cb.to_df()
    display(logs_df)

    # Save artifacts
    def _slug(s: str) -> str:
        return re.sub(r'[^A-Za-z0-9_\-]+', '_', s.replace('/', '-')).strip('_')
    base = f"{_slug(student_model_name)}_distilled_from_{_slug(distilled_from)}"
    os.makedirs(base, exist_ok=True)
    student_final.save_pretrained(base, safe_serialization=True)
    tokenizer_student.save_pretrained(base)
    torch.save(student_final.state_dict(), f"{base}.pt")

    # Evaluate on test with the student tokenizer (single-input dataset)
    from torch.nn import CrossEntropyLoss
    from torch.utils.data import DataLoader
    test_loader = DataLoader(
        make_hf_dataset(df_test, tokenizer_student, text_col, label_col, max_len, torch_format=True),
        batch_size=best["batch_size"], shuffle=False
    )
    eval_metrics = evaluate_model(student_final, test_loader, CrossEntropyLoss(), device=device)
    return student_final, eval_metrics

For each fine-tuned model we will use a different student model:
- BERTweet teacher → BERT-tiny student
- DeBERTa teacher → DeBERTa small student

In [50]:
# Hugging Face saved teacher models
bertweet_hf_path = "vinai-bertweet-base_fine_tuned_hf_model.pt"
deberta_hf_path  = "microsoft-deberta-base_fine_tuned_hf_model.pt"

# PyTorch saved teacher models
bertweet_pt_path = "vinai-bertweet-base_fine_tuned_pytorch_model.pt"
deberta_pt_path  = "microsoft-deberta-base_fine_tuned_pytorch_model.pt"

In [ ]:
# Define a list to store the results
kd_results = []

# Define a config list with all 4 setups
kd_configs = [
    {
        "teacher_name": "vinai/bertweet-base",
        "teacher_path": bertweet_hf_path,
        "student_name": "prajjwal1/bert-tiny",
        "label": "BERTweet-HF Teacher"
    },
    {
        "teacher_name": "vinai/bertweet-base",
        "teacher_path": bertweet_pt_path,
        "student_name": "prajjwal1/bert-tiny",
        "label": "BERTweet-PyTorch Teacher"
    },
    {
        "teacher_name": "microsoft/deberta-base",
        "teacher_path": deberta_hf_path,
        "student_name": "microsoft/deberta-v3-small",
        "label": "DeBERTa-HF Teacher"
    },
    {
        "teacher_name": "microsoft/deberta-base",
        "teacher_path": deberta_pt_path,
        "student_name": "microsoft/deberta-v3-small",
        "label": "DeBERTa-PyTorch Teacher"
    }
]

# Loop through configs and run KD
for cfg in kd_configs:
    print(f"\nRunning KD with {cfg['label']}")

    student_tok = AutoTokenizer.from_pretrained(cfg["student_name"], use_fast=True)
    teacher_tok = AutoTokenizer.from_pretrained(cfg["teacher_name"], use_fast=True)

    student_model, student_metrics = distill_with_optuna_and_evaluation(
        teacher_model_name=cfg["teacher_name"],
        teacher_state_dict_path=cfg["teacher_path"],
        student_model_name=cfg["student_name"],
        tokenizer_student=student_tok,
        tokenizer_teacher=teacher_tok,
        df_train=df_train,
        df_test=df_test,
        text_col="TweetWithDateLocation",
        label_col="label",
        max_len=128,
        device=device,
        n_trials=3,
        n_splits=3,
        output_dir=".",
        distilled_from=cfg["label"]
    )

    kd_results.append({"Teacher": cfg["label"], "Student": cfg["student_name"].replace("/", "-"), **student_metrics})

kd_df = pd.DataFrame(kd_results)
display(kd_df)


Running KD with BERTweet-HF Teacher


Map:   0%|          | 0/27434 [00:00<?, ? examples/s]

Map:   0%|          | 0/13718 [00:00<?, ? examples/s]

Map:   0%|          | 0/27435 [00:00<?, ? examples/s]

Map:   0%|          | 0/13717 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/41152 [00:00<?, ? examples/s]

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

,step,epoch,loss,grad_norm,learning_rate,eval_loss,eval_accuracy,eval_macro_f1,eval_macro_precision,eval_macro_recall,eval_runtime,eval_samples_per_second,eval_steps_per_second,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,2572,1.0,0.7096,1.325137,8.379841e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2572,1.0,NaN,NaN,NaN,0.593645,0.774882,0.785262,0.787001,0.792577,12.5792,301.928,18.920,NaN,NaN,NaN,NaN,NaN
2,5144,2.0,0.4579,2.827646,5.588370e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5144,2.0,NaN,NaN,NaN,0.548257,0.810953,0.821066,0.825025,0.826349,12.5859,301.765,18.910,NaN,NaN,NaN,NaN,NaN
4,7716,3.0,0.3759,2.578553,2.796899e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7716,3.0,NaN,NaN,NaN,0.539781,0.842812,0.850058,0.852876,0.851733,12.5434,302.789,18.974,NaN,NaN,NaN,NaN,NaN
6,10288,4.0,0.3219,1.826887,4.343011e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10288,4.0,NaN,NaN,NaN,0.512489,0.850974,0.857699,0.860681,0.857237,12.6439,300.381,18.823,NaN,NaN,NaN,NaN,NaN
8,10288,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,905.4678,181.793,11.362,0.0,0.466324


Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

,Teacher,Student,Loss,F1 Score (macro),Accuracy,Precision (macro),Recall (macro),Inference Time (sec),Inference Time (sec/sample),Model Size (Mb),Parameter Count
0,BERTweet-HF Teacher,prajjwal1-bert-tiny,0.722356,0.747749,0.735387,0.750988,0.747017,0.651365,0.000172,17.563094,4386565
1,BERTweet-PyTorch Teacher,prajjwal1-bert-tiny,0.714201,0.749952,0.736177,0.754921,0.748318,1.107113,0.000291,17.563094,4386565
2,DeBERTa-HF Teacher,microsoft-deberta-v3-small,0.417227,0.853924,0.846761,0.854198,0.857512,2.008287,0.000529,567.639904,141898757
3,DeBERTa-PyTorch Teacher,microsoft-deberta-v3-small,0.454154,0.857699,0.850974,0.860681,0.857237,3.900842,0.001027,567.639904,141898757
